<a href="https://colab.research.google.com/github/RCortez25/PhD/blob/main/LLM/4.%20Attention%20mechanism/3_Multi_head_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> In Multi head attention, one divides the attention mechanism into multiple heads, each operating independently.
>
> One stacks multiple single head attention layers. That is, one creates multiple instances of the self-attention mechanism, each with its own weights and then they are combined.
>
> Even though this is computationally expensive, it allows the LLM to capture complex patterns.

In [ ]:
class MultiHeadAttentionWrapper(nn.Module):
    pass